In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np


image = tf.placeholder(tf.float32, shape=[None, 500, 500, 1], name="image")
real_shape = tf.placeholder(tf.float32, shape=[None, 4], name='shape')


layer = tf.layers.conv2d(image, 10, 5, strides=2, padding='valid')
layer = tf.nn.relu(layer)
layer = tf.layers.average_pooling2d(layer, 3, 2)
layer = tf.layers.dropout(layer)
layer = tf.layers.conv2d(image, 20, 21, strides=10, padding='valid')
layer = tf.nn.relu(layer)
layer = tf.layers.average_pooling2d(layer, 3, 2)
layer = tf.layers.dropout(layer)
layer = tf.layers.flatten(layer)
layer = tf.layers.dense(layer, 50)
layer = tf.nn.relu(layer)
logits = tf.layers.dense(layer, 4)
output = tf.nn.softmax(logits, name="output")

saver = tf.train.Saver()

In [2]:
sess=tf.Session()
# Restore variables from disk.
tf.train.Saver().restore(sess, "/tmp_formas/model.ckpt")
image0 = tf.get_default_graph().get_tensor_by_name("image:0")
output0 = tf.get_default_graph().get_tensor_by_name("output:0")

INFO:tensorflow:Restoring parameters from /tmp_formas/model.ckpt


In [3]:
import pygame
from pygame.locals import *
import numpy as np


pygame.init()
screen=pygame.display.set_mode((500, 500),pygame.RESIZABLE|DOUBLEBUF)
screen_on = True
pygame.font.init()
myfont = pygame.font.SysFont('Comic Sans MS', 20)
textSquare= myfont.render('Cuadrado', False, (0,0,0))
textCircle= myfont.render('Circulo', False, (0,0,0))
textTriangle= myfont.render('Triangulo', False, (0,0,0))
textStar= myfont.render('Estrella', False, (0,0,0))
textShapes = [textSquare, textCircle, textTriangle, textStar]

def square(screen, x, y, s):
    pygame.draw.rect(screen, (0,0,0), (x-s/2, y-s/2, s, s))
def circle(screen, x, y, s):
    pygame.draw.circle(screen, (0,0,0), (x, y), int(s*0.66))
def triangle(screen, x, y, s):
    p1= (x, y+s/2)
    p2= (x-s/2, y-s/2)
    p3= (x + s/2, y-s/2)
    pygame.draw.polygon(screen, (0,0,0), (p1, p2, p3))
def star(screen, x, y, s):
    n=5
    q=3
    p = [(x+s*np.sin(i*2*np.pi/n), y-s*np.cos(i*2*np.pi/n)) for i in range(n)]
    order = [p[(q*i)%n] for i in range(n)]
    pygame.draw.polygon(screen, (0,0,0), order)
shapes=[square, circle, triangle, star]
r1=np.random.randint(0, high=len(shapes))
r2=np.random.randint(20, high=200)
r3=np.random.randint(r2, high=500-r2)
r4=np.random.randint(r2, high=500-r2)
prediccion=0


while screen_on:
    pygame.draw.rect(screen, (255,255,255), (0, 0, 500, 500))
    shapes[r1](screen, r3, r4, r2)
    prediccion=sess.run(output0, feed_dict={image0: np.array(pygame.surfarray.array2d(screen)).reshape([-1,500,500,1])/16777215})
    screen.blit(textShapes[np.argmax(prediccion[0])], (50, 50))
    pygame.display.flip()
    for event in pygame.event.get():

        if event.type == pygame.KEYDOWN:
            if event.key == K_ESCAPE:
                pygame.quit()
                screen_on = False
        if event.type == pygame.MOUSEBUTTONDOWN:
            r1=np.random.randint(0, high=len(shapes))
            r2=np.random.randint(20, high=200)
            r3=np.random.randint(r2, high=500-r2)
            r4=np.random.randint(r2, high=500-r2)
            
            